<a href="https://colab.research.google.com/github/AsifAlFaisal/weather-data-comparison/blob/master/Historical_vs_Observed_Temperature_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Author:** ©*Asif Al Faisal*

**Email:** asif.faisal.eee@gmail.com

**Organization:** CIMMYT

In [0]:
import pandas as pd
import calendar
import plotly.offline as pyo
import plotly.graph_objs as go

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
tmax_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CIMMYT/191219/Tmax_BMD_final.csv')
tmin_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CIMMYT/191219/Tmin_BMD_final.csv')
#obs_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CIMMYT/191202/jas18.csv')
din_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CIMMYT/191219/din19.csv')
raj_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CIMMYT/191219/raj19.csv')
bho_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CIMMYT/191219/bhola19.csv')

In [0]:
obs_df.columns = ['Year', 'Month', 'Day', 'ObsMax', 'ObsMin']

In [0]:
tmin_jas = tmin_df[['Year','Month','Day','Jashore']]
tmax_jas = tmax_df[['Jashore']]
tmin_jas.columns = ['Year','Month','Day','Min']
tmax_jas.columns = ['Max']

In [0]:
history_jas = pd.concat([tmin_jas,tmax_jas], axis=1, sort=False)

In [0]:
history_jas.head()

,Year,Month,Day,Min,Max
0,1984,1,1,7.3,23.6
1,1984,1,2,9.5,28.0
2,1984,1,3,15.0,24.0
3,1984,1,4,12.2,25.0
4,1984,1,5,11.0,24.6


In [0]:
history_jas = history_jas[history_jas['Min']!=-9999.0]

In [0]:
history_jas['Year'] = history_jas['Year'].astype('int64')
history_jas['Month'] = history_jas['Month'].astype('int64')

In [0]:
nov = history_jas[history_jas['Month']==11].groupby(['Day']).mean()
nov['minSE'] = history_jas[history_jas['Month']==11].groupby(['Day']).std()['Min']
nov['maxSE'] = history_jas[history_jas['Month']==11].groupby(['Day']).std()['Max']
nov = nov.reset_index()
nov = nov[['Day','Month','Min','Max','minSE','maxSE']]
nov['Month'] = nov['Month'].astype('int64')


dec = history_jas[history_jas['Month']==12].groupby(['Day']).mean()
dec['minSE'] = history_jas[history_jas['Month']==12].groupby(['Day']).std()['Min']
dec['maxSE'] = history_jas[history_jas['Month']==12].groupby(['Day']).std()['Max']
dec = dec.reset_index()
dec = dec[['Day','Month','Min','Max','minSE','maxSE']]
dec['Month'] = dec['Month'].astype('int64')


jan = history_jas[history_jas['Month']==1].groupby(['Day']).mean()
jan['minSE'] = history_jas[history_jas['Month']==1].groupby(['Day']).std()['Min']
jan['maxSE'] = history_jas[history_jas['Month']==1].groupby(['Day']).std()['Max']
jan = jan.reset_index()
jan = jan[['Day','Month','Min','Max','minSE','maxSE']]
jan['Month'] = jan['Month'].astype('int64')

feb = history_jas[history_jas['Month']==2].groupby(['Day']).mean()
feb['minSE'] = history_jas[history_jas['Month']==2].groupby(['Day']).std()['Min']
feb['maxSE'] = history_jas[history_jas['Month']==2].groupby(['Day']).std()['Max']
feb = feb.reset_index()
feb = feb[['Day','Month','Min','Max','minSE','maxSE']]
feb['Month'] = feb['Month'].astype('int64')

In [0]:
tot_hist = nov.append(dec).append(jan).append(feb)

In [0]:
tot_hist = tot_hist.reset_index()

In [0]:

tot_hist['Month'] = tot_hist['Month'].apply(lambda x: calendar.month_abbr[x])
tot_hist["comb"] = tot_hist["Month"] +"-"+tot_hist["Day"].map(str) 

In [0]:
his_obs = pd.concat([tot_hist,obs_df[['ObsMax','ObsMin']]], axis=1, sort=False)

In [0]:
trace1 = go.Scatter(
    x = his_obs.comb,
    y = his_obs.Max,
    mode = 'lines + markers',
    name='Historical Max Temperature (1984-2017)',
    error_y=dict(
        type='data',
        array=tot_hist.maxSE,
        visible = True,
        thickness = 0.5 
    )
)
trace2 = go.Scatter(
    x = his_obs.comb,
    y = his_obs.Min,
    mode = 'lines + markers',
    name='Historical Min Temperature (1984-2017)',
    error_y=dict(
        type='data',
        array=tot_hist.minSE,
        visible = True,
        thickness = 0.5 
    )
)

trace3 = go.Scatter(
    x = his_obs.comb,
    y = his_obs.ObsMax,
    mode = 'lines + markers',
    marker = dict(size=8),
    name='Observed Max Temperature - 2018',
)
trace4 = go.Scatter(
    x = his_obs.comb,
    y = his_obs.ObsMin,
    mode = 'lines + markers',
    marker = dict(size=8),
    name='Observed Min Temperature - 2018',
)


data = [trace1, trace2, trace3, trace4]

layout = go.Layout(
    title = 'Jashore Station Historic vs Observed Temperature',
    xaxis = dict(title='Days'),
    yaxis = dict(title='Temperature'),
    hovermode = 'closest',
)

fig = go.Figure(data=data, layout = layout)
fig.show()

In [0]:
fig.write_html('/content/drive/My Drive/Colab Notebooks/CIMMYT/191202/demo_jashore.html')

# Dinajpur

In [0]:
tmin_din = tmin_df[['Year','Month','Day','Dinajpur']]
tmax_din = tmax_df[['Dinajpur']]
tmin_din.columns = ['Year','Month','Day','Min']
tmax_din.columns = ['Max']
history_din = pd.concat([tmin_din,tmax_din], axis=1, sort=False)

In [0]:
history_din.tail()

,Year,Month,Day,Min,Max
12779,2018,12,27,8.3,24.2
12780,2018,12,28,8.0,22.2
12781,2018,12,29,6.0,23.7
12782,2018,12,30,7.1,25.2
12783,2018,12,31,7.5,25.5


In [0]:
history_din['Year'] = history_din['Year'].astype('int64')
history_din['Month'] = history_din['Month'].astype('int64')

In [0]:
novd = history_din[history_din['Month']==11].groupby(['Day']).mean()
novd['minSE'] = history_din[history_din['Month']==11].groupby(['Day']).std()['Min']
novd['maxSE'] = history_din[history_din['Month']==11].groupby(['Day']).std()['Max']
novd = novd.reset_index()
novd = novd[['Day','Month','Min','Max','minSE','maxSE']]
novd['Month'] = novd['Month'].astype('int64')


decd = history_din[history_din['Month']==12].groupby(['Day']).mean()
decd['minSE'] = history_din[history_din['Month']==12].groupby(['Day']).std()['Min']
decd['maxSE'] = history_din[history_din['Month']==12].groupby(['Day']).std()['Max']
decd = decd.reset_index()
decd = decd[['Day','Month','Min','Max','minSE','maxSE']]
decd['Month'] = decd['Month'].astype('int64')


jand = history_din[history_din['Month']==1].groupby(['Day']).mean()
jand['minSE'] = history_din[history_din['Month']==1].groupby(['Day']).std()['Min']
jand['maxSE'] = history_din[history_din['Month']==1].groupby(['Day']).std()['Max']
jand = jand.reset_index()
jand = jand[['Day','Month','Min','Max','minSE','maxSE']]
jand['Month'] = jand['Month'].astype('int64')

febd = history_din[history_din['Month']==2].groupby(['Day']).mean()
febd['minSE'] = history_din[history_din['Month']==2].groupby(['Day']).std()['Min']
febd['maxSE'] = history_din[history_din['Month']==2].groupby(['Day']).std()['Max']
febd = febd.reset_index()
febd = febd[['Day','Month','Min','Max','minSE','maxSE']]
febd['Month'] = febd['Month'].astype('int64')

In [0]:
tot_histd = novd.append(decd).append(jand).append(febd)
tot_histd = tot_histd.reset_index()

In [0]:
tot_histd['Month'] = tot_histd['Month'].apply(lambda x: calendar.month_abbr[x])
tot_histd["comb"] = tot_histd["Month"] +"-"+tot_histd["Day"].map(str) 

In [0]:
his_obsd = pd.concat([tot_histd,din_df[['obsMax','obsMin']]], axis=1, sort=False)

In [0]:
his_obsd.tail()

,index,Day,Month,Min,Max,minSE,maxSE,comb,obsMax,obsMin
116,24,25,Feb,14.914286,28.071429,2.633064,2.213936,Feb-25,NaN,NaN
117,25,26,Feb,14.780000,28.251429,3.052848,2.068158,Feb-26,NaN,NaN
118,26,27,Feb,15.042857,28.105714,2.693160,1.914333,Feb-27,NaN,NaN
119,27,28,Feb,14.948571,28.562857,2.122896,1.761197,Feb-28,NaN,NaN
120,28,29,Feb,13.944444,28.544444,2.184097,2.231654,Feb-29,NaN,NaN


In [0]:
trace1 = go.Scatter(
    x = his_obsd.comb,
    y = his_obsd.Max,
    mode = 'lines + markers',
    name='Historical Max Temperature (1984-2018)',
    error_y=dict(
        type='data',
        array=tot_histd.maxSE,
        visible = True,
        thickness = 0.5 
    )
)
trace2 = go.Scatter(
    x = his_obsd.comb,
    y = his_obsd.Min,
    mode = 'lines + markers',
    name='Historical Min Temperature (1984-2018)',
    error_y=dict(
        type='data',
        array=tot_histd.minSE,
        visible = True,
        thickness = 0.5 
    )
)

trace3 = go.Scatter(
    x = his_obsd.comb,
    y = his_obsd.obsMax,
    mode = 'lines + markers',
    marker = dict(size=8),
    name='Observed Max Temperature (2019)',
)
trace4 = go.Scatter(
    x = his_obsd.comb,
    y = his_obsd.obsMin,
    mode = 'lines + markers',
    marker = dict(size=8),
    name='Observed Min Temperature (2019)',
)


data = [trace1, trace2, trace3, trace4]

layout = go.Layout(
    title = 'Dinajpur Station Historic vs Observed Temperature',
    xaxis = dict(title='Days'),
    yaxis = dict(title='Temperature'),
    hovermode = 'closest',
)

fig = go.Figure(data=data, layout = layout)
fig.show()

In [0]:
fig.write_html('/content/drive/My Drive/Colab Notebooks/CIMMYT/191219/191219_dinajpur(BMDvsKOBO).html')

In [0]:
tmin_raj = tmin_df[['Year','Month','Day','Rajshahi']]
tmax_raj = tmax_df[['Rajshahi']]
tmin_raj.columns = ['Year','Month','Day','Min']
tmax_raj.columns = ['Max']
history_raj = pd.concat([tmin_raj,tmax_raj], axis=1, sort=False)

#history_raj = history_raj[history_jas['Min']!=-9999.0]
#history_raj = history_raj[history_jas['Max']!=-9999.0]

history_raj['Year'] = history_raj['Year'].astype('int64')
history_raj['Month'] = history_raj['Month'].astype('int64')

novr = history_raj[history_raj['Month']==11].groupby(['Day']).mean()
novr['minSE'] = history_raj[history_raj['Month']==11].groupby(['Day']).std()['Min']
novr['maxSE'] = history_raj[history_raj['Month']==11].groupby(['Day']).std()['Max']
novr = novr.reset_index()
novr = novr[['Day','Month','Min','Max','minSE','maxSE']]
novr['Month'] = novr['Month'].astype('int64')


decr = history_raj[history_raj['Month']==12].groupby(['Day']).mean()
decr['minSE'] = history_raj[history_raj['Month']==12].groupby(['Day']).std()['Min']
decr['maxSE'] = history_raj[history_raj['Month']==12].groupby(['Day']).std()['Max']
decr = decr.reset_index()
decr = decr[['Day','Month','Min','Max','minSE','maxSE']]
decr['Month'] = decr['Month'].astype('int64')


janr = history_raj[history_raj['Month']==1].groupby(['Day']).mean()
janr['minSE'] = history_raj[history_raj['Month']==1].groupby(['Day']).std()['Min']
janr['maxSE'] = history_raj[history_raj['Month']==1].groupby(['Day']).std()['Max']
janr = janr.reset_index()
janr = janr[['Day','Month','Min','Max','minSE','maxSE']]
janr['Month'] = janr['Month'].astype('int64')

febr = history_raj[history_raj['Month']==2].groupby(['Day']).mean()
febr['minSE'] = history_raj[history_raj['Month']==2].groupby(['Day']).std()['Min']
febr['maxSE'] = history_raj[history_raj['Month']==2].groupby(['Day']).std()['Max']
febr = febr.reset_index()
febr = febr[['Day','Month','Min','Max','minSE','maxSE']]
febr['Month'] = febr['Month'].astype('int64')

tot_histr = novr.append(decr).append(janr).append(febr)
tot_histr = tot_histr.reset_index()

tot_histr['Month'] = tot_histr['Month'].apply(lambda x: calendar.month_abbr[x])
tot_histr["comb"] = tot_histr["Month"] +"-"+tot_histr["Day"].map(str) 


his_obsr = pd.concat([tot_histr,raj_df[['obsMax','obsMin']]], axis=1, sort=False)

In [0]:
his_obsr.tail()

,index,Day,Month,Min,Max,minSE,maxSE,comb,obsMax,obsMin
116,24,25,Feb,14.782857,29.714286,3.238145,2.700451,Feb-25,NaN,NaN
117,25,26,Feb,14.962857,29.842857,2.792541,2.604327,Feb-26,NaN,NaN
118,26,27,Feb,14.988571,29.694286,3.067646,2.321883,Feb-27,NaN,NaN
119,27,28,Feb,14.688571,30.334286,2.489835,2.110470,Feb-28,NaN,NaN
120,28,29,Feb,14.111111,30.311111,3.286124,2.304585,Feb-29,NaN,NaN


In [0]:
trace1 = go.Scatter(
    x = his_obsr.comb,
    y = his_obsr.Max,
    mode = 'lines + markers',
    name='Historical Max Temperature (1984-2018)',
    error_y=dict(
        type='data',
        array=tot_histr.maxSE,
        visible = True,
        thickness = 0.5 
    )
)
trace2 = go.Scatter(
    x = his_obsr.comb,
    y = his_obsr.Min,
    mode = 'lines + markers',
    name='Historical Min Temperature (1984-2018)',
    error_y=dict(
        type='data',
        array=tot_histr.minSE,
        visible = True,
        thickness = 0.5 
    )
)

trace3 = go.Scatter(
    x = his_obsr.comb,
    y = his_obsr.obsMax,
    mode = 'lines + markers',
    marker = dict(size=8),
    name='Observed Max Temperature (2019)',
)
trace4 = go.Scatter(
    x = his_obsr.comb,
    y = his_obsr.obsMin,
    mode = 'lines + markers',
    marker = dict(size=8),
    name='Observed Min Temperature (2019)',
)


data = [trace1, trace2, trace3, trace4]

layout = go.Layout(
    title = 'Rajshahi Station Historic vs Observed Temperature',
    xaxis = dict(title='Days'),
    yaxis = dict(title='Temperature'),
    hovermode = 'closest',
)

fig = go.Figure(data=data, layout = layout)
fig.show()

In [0]:
fig.write_html('/content/drive/My Drive/Colab Notebooks/CIMMYT/191219/191219_rajshahi(BMDvsKOBO).html')

In [0]:
tmin_bho = tmin_df[['Year','Month','Day','Bhola']]
tmax_bho = tmax_df[['Bhola']]
tmin_bho.columns = ['Year','Month','Day','Min']
tmax_bho.columns = ['Max']
history_bho = pd.concat([tmin_bho,tmax_bho], axis=1, sort=False)

history_bho = history_bho[history_bho['Min']!=-9999.0]
history_bho = history_bho[history_bho['Max']!=-9999.0]

history_bho['Year'] = history_bho['Year'].astype('int64')
history_bho['Month'] = history_bho['Month'].astype('int64')

novb = history_bho[history_bho['Month']==11].groupby(['Day']).mean()
novb['minSE'] = history_bho[history_bho['Month']==11].groupby(['Day']).std()['Min']
novb['maxSE'] = history_bho[history_bho['Month']==11].groupby(['Day']).std()['Max']
novb = novb.reset_index()
novb = novb[['Day','Month','Min','Max','minSE','maxSE']]
novb['Month'] = novb['Month'].astype('int64')


decb = history_bho[history_bho['Month']==12].groupby(['Day']).mean()
decb['minSE'] = history_bho[history_bho['Month']==12].groupby(['Day']).std()['Min']
decb['maxSE'] = history_bho[history_bho['Month']==12].groupby(['Day']).std()['Max']
decb = decb.reset_index()
decb = decb[['Day','Month','Min','Max','minSE','maxSE']]
decb['Month'] = decb['Month'].astype('int64')


janb = history_bho[history_bho['Month']==1].groupby(['Day']).mean()
janb['minSE'] = history_bho[history_bho['Month']==1].groupby(['Day']).std()['Min']
janb['maxSE'] = history_bho[history_bho['Month']==1].groupby(['Day']).std()['Max']
janb = janb.reset_index()
janb = janb[['Day','Month','Min','Max','minSE','maxSE']]
janb['Month'] = janb['Month'].astype('int64')

febb = history_bho[history_bho['Month']==2].groupby(['Day']).mean()
febb['minSE'] = history_bho[history_bho['Month']==2].groupby(['Day']).std()['Min']
febb['maxSE'] = history_bho[history_bho['Month']==2].groupby(['Day']).std()['Max']
febb = febb.reset_index()
febb = febb[['Day','Month','Min','Max','minSE','maxSE']]
febb['Month'] = febb['Month'].astype('int64')

tot_histb = novb.append(decb).append(janb).append(febb)
tot_histb = tot_histb.reset_index()

tot_histb['Month'] = tot_histb['Month'].apply(lambda x: calendar.month_abbr[x])
tot_histb["comb"] = tot_histb["Month"] +"-"+tot_histb["Day"].map(str) 


In [0]:
his_obsb = pd.concat([tot_histb,bho_df[['obsMax','obsMin']]], axis=1, sort=False)
his_obsb.tail()

,index,Day,Month,Min,Max,minSE,maxSE,comb,obsMax,obsMin
116,24,25,Feb,17.071429,30.020000,3.245410,2.316158,Feb-25,NaN,NaN
117,25,26,Feb,18.260000,30.048571,3.421403,2.142891,Feb-26,NaN,NaN
118,26,27,Feb,18.120000,29.894286,3.204390,1.836067,Feb-27,NaN,NaN
119,27,28,Feb,17.954286,30.462857,2.811985,1.455635,Feb-28,NaN,NaN
120,28,29,Feb,17.688889,29.755556,3.349420,1.618727,Feb-29,NaN,NaN


In [0]:
trace1 = go.Scatter(
    x = his_obsb.comb,
    y = his_obsb.Max,
    mode = 'lines + markers',
    name='Historical Max Temperature (1984-2018)',
    error_y=dict(
        type='data',
        array=tot_histb.maxSE,
        visible = True,
        thickness = 0.5 
    )
)
trace2 = go.Scatter(
    x = his_obsb.comb,
    y = his_obsb.Min,
    mode = 'lines + markers',
    name='Historical Min Temperature (1984-2018)',
    error_y=dict(
        type='data',
        array=tot_histb.minSE,
        visible = True,
        thickness = 0.5 
    )
)

trace3 = go.Scatter(
    x = his_obsb.comb,
    y = his_obsb.obsMax,
    mode = 'lines + markers',
    marker = dict(size=8),
    name='Observed Max Temperature (2019)',
)
trace4 = go.Scatter(
    x = his_obsb.comb,
    y = his_obsb.obsMin,
    mode = 'lines + markers',
    marker = dict(size=8),
    name='Observed Min Temperature (2019)',
)


data = [trace1, trace2, trace3, trace4]

layout = go.Layout(
    title = 'Bhola Station Historic vs Observed Temperature',
    xaxis = dict(title='Days'),
    yaxis = dict(title='Temperature'),
    hovermode = 'closest',
)

fig = go.Figure(data=data, layout = layout)
fig.show()

In [0]:
fig.write_html('/content/drive/My Drive/Colab Notebooks/CIMMYT/191219/191219_bhola(BMDvsKOBO).html')